# Setup

## Imports

In [55]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt

## Fetch Data

In [41]:
concrete_header = ["Cement", "Slag", "Fly Ash", "Water", "SP", "Coarse Aggr", "Fine Aggr", "SLUMP Flow"]
train_concrete = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/concrete/train.csv", names=concrete_header)
test_concrete = pd.read_csv("https://raw.githubusercontent.com/DoubekSeth/ToyDatasets/main/concrete/test.csv", names=concrete_header)

# Algorithm

Method that adds ones to a dataframe, makes algorithm simplier to implement

In [42]:
def addOnesForBiasOnDataset(df):
    rows = df.shape[0]
    columns = df.shape[1]
    ones = np.ones(rows)
    #Need to check if actually need to insert
    if("Bias Term" not in df.columns):
        df.insert(columns-1, "Bias Term", ones, False)

In [43]:
def BatchGD(train_df, gradient_of_cost_func, r, weights, convergence):
    converged = False
    t=0
    costs=[]
    while(not converged):
        grad = gradient_of_cost_func(train_df, weights)
        #print(grad)
        new_weights = weights - r*grad
        if(abs(np.sum(weights-new_weights)) < convergence):
            converged=True
        weights = new_weights
        #Print cost
        cost = cost_MSE_df(weights, train_df)
        t+=1
        print("Cost at step", t, ":", cost)
        costs.append(cost)
    return weights, costs, t

Returns the gradient of the training data with the weights for a MSE cost function

In [44]:
def gradient_Batch_MSE(train_df, weights):
    grad = np.zeros(train_df.shape[1]-1) #Subtract one for the label
    X = train_df.drop("SLUMP Flow", axis=1)
    Y = train_df["SLUMP Flow"]
    for i in range(len(grad)):
        X_i = train_df.iloc[:, i]
        #print("WTX", np.dot(X, weights))
        #print("Y-WTX", Y-np.dot(X, weights))
        #print("X_i", X_i)
        #print("-(Y-WTX)X_i", -np.dot((Y-np.dot(X, weights)), X_i))
        grad[i] = -np.dot((Y-np.dot(X, weights)), X_i)

    return grad

In [128]:
def StochasticGD(train_df, r, weights, convergence):
    converged = False
    t=0
    costs=[]
    while(not converged):
        for index, example in train_df.iterrows():
            new_weights = copy.copy(weights)
            for j in range(len(weights)):
                #print(train_df.iloc[index, train_df.shape[1]-1])
                #print(np.dot(example.drop("SLUMP Flow"), weights))
                #print(train_df.iloc[index, j])
                new_weights[j] = weights[j] + r*(train_df.iloc[index, train_df.shape[1]-1]-np.dot(example.drop("SLUMP Flow"), weights)*train_df.iloc[index, j])            
            if(abs(np.sum(weights-new_weights)) < convergence):
                converged=True
            weights=new_weights
            t+=1
            cost = cost_MSE_df(weights, train_df)
            print("Cost at step", t, ":", cost)
            costs.append(cost)
    return weights, costs, t

In [46]:
def cost_MSE_df(weights, data):
    X = data.drop("SLUMP Flow", axis=1)
    Y = data["SLUMP Flow"]
    return 0.5*(np.sum(np.square(Y-np.dot(X, weights))))

In [47]:
def cost_MSE(weights, datum):
    return 0.5*(datum["SLUMP Flow"]-np.dot(weights, datum.drop("SLUMP Flow")))

# Accuracy Evaluations

weights = np.zeros(8)
addOnesForBiasOnDataset(train_concrete)
#print(gradient_Batch_MSE(train_concrete, weights))
weights, costs, steps = BatchGD(train_concrete, gradient_Batch_MSE, 0.005, weights, .000001)
plt.plot(np.linspace(1, steps, steps), costs, marker='o', linestyle='-')
plt.xlabel("Steps")
plt.ylabel("Cost")
plt.grid(True)
plt.show
print(weights)

addOnesForBiasOnDataset(test_concrete)
cost_MSE_df(weights, test_concrete)

weights = np.zeros(8)
addOnesForBiasOnDataset(train_concrete)
#print(gradient_Batch_MSE(train_concrete, weights))
weights, costs, steps = StochasticGD(train_concrete, 0.02, weights, .00002)
plt.plot(np.linspace(1, steps, steps), costs, marker='o', linestyle='-')
plt.xlabel("Steps")
plt.ylabel("Cost")
plt.grid(True)
plt.show
print(weights)
addOnesForBiasOnDataset(test_concrete)
print(cost_MSE_df(weights, test_concrete))

weights, residuals, rank, s = np.linalg.lstsq(train_concrete.drop("SLUMP Flow", axis=1), train_concrete["SLUMP Flow"])
print(weights)
cost_MSE_df(weights, test_concrete)

paper_df = pd.DataFrame([[1, -1, 2, 1], [1, 1, 3, 4], [-1, 1, 0, -1], [1, 2, -4, -2], [3, -1, -1, 0]])
paper_df.columns = ["x_1", "x_2", "x_3", "SLUMP Flow"]
addOnesForBiasOnDataset(paper_df)
print(paper_df)

StochasticGD(paper_df, r=0.1, weights=np.zeros(4), convergence=0.001)